In [37]:
#import packages
import numpy as np
from random import random, seed
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [38]:
#define Franke function
def FrankeFunction(x,y): #still got one division in here
    term1 = 0.75*np.exp(-(0.25*(9*x-2)*(9*x-2)) - 0.25*((9*y-2)*(9*y-2)))
    term2 = 0.75*np.exp(-((9*x+1)*(9*x+1))/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)*(9*x-7)*0.25 - 0.25*((9*y-3)*(9*y-3)))
    term4 = -0.2*np.exp(-(9*x-4)*(9*x-4) - (9*y-7)*(9*y-7))
    return term1 + term2 + term3 + term4

In [39]:
#create model matrix
# Form is: x0y0, x1y0, x2y0, x0y1, x1y1, x0y2
def Model(x,y,P): #P is polynomial degree
    x = np.array(x)
    y = np.array(y) #trengs disse to?
    m = len(x) #vector length
    t = sum(range(P+2)) # number of terms in polynomial
    X = np.zeros((m,t)) # Model matrix
    c = 0 #counter
    for i in range(P+1):
        for j in range(P+1-i):
            X[:,c] = x**j*y**i
            c +=1
    return X

In [40]:
#define R2 function
def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_model)) ** 2)

In [41]:
#define MSE function
def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

In [42]:
#define variance function
def Var(y_data, y_model,P,X):
    N = len(y_data)
    covar = np.linalg.inv(X.T.dot(X))#sigma2
    vari = np.diagonal(covar)
    return vari

In [43]:
#Create random variables/predictors
np.random.seed(1234)
N = 100 #points along x and y axes

x = np.random.uniform(0,1,N)
y = np.random.uniform(0,1,N)

Noise = 0.1*np.random.randn(N) #ask how noise should be added. NxN?

#create datapoints/results
z = FrankeFunction(x,y)

#add noise
z_n = z+Noise

In [44]:
MPD = 2 #maximal polynomial degree

X = Model(x,y,MPD)

In [45]:
# find parameters
beta = np.linalg.inv(X.T.dot(X)).dot(X.T).dot(z_n)

# make prediction
ztilde = X @ beta

In [46]:
######  Makes model matrix, looks different than mine #####
# form: x0y0, x1y0, x0y1, x2y0, x1y1, x0y2
poly2 = PolynomialFeatures(degree=MPD)
a = x[:,np.newaxis]
b = y[:,np.newaxis]
cc = np.c_[a,b]
Xskl = poly2.fit_transform(cc) 
##########################################################
# THIS CELL IS CURRENTLY NOT IN USE #

In [57]:
# find parameters
clf = LinearRegression(fit_intercept=False) #False to not center data, i.e. intercept is not 0
clf.fit(X,z_n) 

#make prediction
zpredict = clf.predict(X)

In [54]:
# The mean squared error              
print("Mean squared error (self): %.5f" % MSE(z_n, ztilde))
print("Mean squared error (skl): %.5f" % mean_squared_error(z_n, zpredict))

# Explained variance score: 1 is perfect prediction     
print('R2 score (self): %.5f' %R2(z_n, ztilde))
print('R2 score (skl): %.5f' % r2_score(z_n, zpredict))

Mean squared error (self): 0.01374
Mean squared error (skl): 0.01374
R2 score (self): 0.79859
R2 score (skl): 0.79859


In [55]:
print(Var(z_n, ztilde, MPD,X))
print(Var(z_n,zpredict,MPD,X))

[0.2716233  2.48506359 1.84460477 2.55413142 1.42486932 2.15716247]
[0.2716233  2.48506359 1.84460477 2.55413142 1.42486932 2.15716247]


In [56]:
print(beta)
print(clf.coef_)

[ 1.10865033 -0.94661259  0.11338428 -0.71930253  0.63430462 -0.22205696]
[ 1.10865033 -0.94661259  0.11338428 -0.71930253  0.63430462 -0.22205696]


In [65]:
std = 1.96*np.sqrt(Var(z_n, ztilde, MPD,X))
print(std)

[1.02150285 3.08976055 2.66199806 3.13240343 2.33961065 2.87870724]


In [64]:
# 95% CI: mu +-1.96*sigma

In [66]:
np.sqrt(0.271633)

0.5211842284643694